**[MDC-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
### Windwos版では下記が必要
import os
###
np.random.seed(20160703)
tf.set_random_seed(20160703)

**[MDC-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
### TensorFlowのバージョンによっては下記のwarningが表示されるが、無視するしかない。
# WARNING:tensorflow:From <ipython-input-2-******>:1: ...
# Instructions for updating:
# Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
#...

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[MDC-03]** フィルターに対応する Variable を用意して、入力データにフィルターとプーリング層を適用する計算式を定義します。

In [3]:
num_filters = 16

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])

W_conv = tf.Variable(tf.truncated_normal([5,5,1,num_filters],
                                         stddev=0.1))
h_conv = tf.nn.conv2d(x_image, W_conv,
                      strides=[1,1,1,1], padding='SAME')
h_pool =tf.nn.max_pool(h_conv, ksize=[1,2,2,1],
                       strides=[1,2,2,1], padding='SAME')

**[MDC-04]** プーリング層からの出力を全結合層を経由してソフトマックス関数に入力する計算式を定義します。

In [4]:
h_pool_flat = tf.reshape(h_pool, [-1, 14*14*num_filters])

num_units1 = 14*14*num_filters
num_units2 = 1024

w2 = tf.Variable(tf.truncated_normal([num_units1, num_units2]))
b2 = tf.Variable(tf.zeros([num_units2]))
hidden2 = tf.nn.relu(tf.matmul(h_pool_flat, w2) + b2)

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
p = tf.nn.softmax(tf.matmul(hidden2, w0) + b0)

**[MDC-05]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [5]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_sum(t * tf.log(p))
train_step = tf.train.AdamOptimizer(0.0005).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[MDC-06]** セッションを用意して、Variable を初期化します。

In [6]:
### >>> with GPU 80%
#gpuConfig = tf.ConfigProto(
#    gpu_options = tf.GPUOptions(
#        per_process_gpu_memory_fraction = 0.8),
#    device_count = {'GPU':0})
#sess = tf.Session(config = gpuConfig)
### <<<
#
### >>> with CPU only
sess = tf.Session()
### <<<
sess.run(tf.global_variables_initializer())
### 下記はPython3.6ではエラーとなるので、上記に変更した。
#sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()
### Windwos版では下記が必要
cwd = os.getcwd()
###

**[MDC-07]** パラメーターの最適化を4000回繰り返します。

最終的に、テストセットに対して約98%の正解率が得られます。

In [7]:
i = 0
for _ in range(4000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, t: batch_ts})
    if i % 100 == 0:
        loss_val, acc_val = sess.run([loss, accuracy],
            feed_dict={x:mnist.test.images, t:mnist.test.labels})
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        ### Windwos版        
        saver.save(sess, cwd + '\\mdc_session', global_step=i)
        ### Linux等、Windows以外であれば、下記行頭の「#」を外し、上記の行頭に「#」を入れる。
        #saver.save(sess, 'mdc_session', global_step=i)

Step: 100, Loss: 2387.759766, Accuracy: 0.928200
Step: 200, Loss: 2044.267822, Accuracy: 0.937100
Step: 300, Loss: 1424.071777, Accuracy: 0.958200
Step: 400, Loss: 1163.766846, Accuracy: 0.963600
Step: 500, Loss: 1061.359375, Accuracy: 0.969400
Step: 600, Loss: 1002.458557, Accuracy: 0.968500
Step: 700, Loss: 1049.682373, Accuracy: 0.969500
Step: 800, Loss: 871.110291, Accuracy: 0.974100
Step: 900, Loss: 842.782227, Accuracy: 0.974000
Step: 1000, Loss: 900.959839, Accuracy: 0.972300
Step: 1100, Loss: 829.934082, Accuracy: 0.974500
Step: 1200, Loss: 881.184448, Accuracy: 0.972800
Step: 1300, Loss: 1018.865967, Accuracy: 0.971500
Step: 1400, Loss: 779.435974, Accuracy: 0.977000
Step: 1500, Loss: 832.644531, Accuracy: 0.974900
Step: 1600, Loss: 719.168579, Accuracy: 0.976900
Step: 1700, Loss: 689.994629, Accuracy: 0.979900
Step: 1800, Loss: 714.230713, Accuracy: 0.977300
Step: 1900, Loss: 661.875732, Accuracy: 0.979700
Step: 2000, Loss: 710.147705, Accuracy: 0.978300
Step: 2100, Loss: 714

**[MDC-08]** セッション情報を保存したファイルが生成されていることを確認します。

In [8]:
### Windows版
!dir mdc_session*
### Linux等、Windows以外であれば、下記行頭の「#」を外し、上記の行頭に「#」を入れる。
#!ls mdc_session*

 ドライブ C のボリューム ラベルがありません。
 ボリューム シリアル番号は xxxx-xxxx です

 C:\Users\xxxx\Jupyter\jupyter_tfbook\Chapter04 のディレクトリ

2019/10/31  18:31        38,675,264 mdc_session-3600.data-00000-of-00001
2019/10/31  18:31               638 mdc_session-3600.index
2019/10/31  18:31            42,615 mdc_session-3600.meta
2019/10/31  18:31        38,675,264 mdc_session-3700.data-00000-of-00001
2019/10/31  18:31               638 mdc_session-3700.index
2019/10/31  18:31            42,615 mdc_session-3700.meta
2019/10/31  18:31        38,675,264 mdc_session-3800.data-00000-of-00001
2019/10/31  18:31               638 mdc_session-3800.index
2019/10/31  18:31            42,615 mdc_session-3800.meta
2019/10/31  18:31        38,675,264 mdc_session-3900.data-00000-of-00001
2019/10/31  18:31               638 mdc_session-3900.index
2019/10/31  18:31            42,615 mdc_session-3900.meta
2019/10/31  18:31        38,675,264 mdc_session-4000.data-00000-of-00001
2019/10/31  18:31               638 mdc_session-4000.in